# 10-20RMSE&20-50RMSE

## 观察数据情况

### 变量缺失率情况

In [5]:
import pandas as pd

# 1. 读取数据
file_path = r'C:\Users\phc\Desktop\IamDataFrame2010_2100.csv'
df = pd.read_csv(file_path)

# 2. 获取 Variable 的唯一数量
unique_variables = df['Variable'].nunique()
print(f"Variable 列中总共有 {unique_variables} 个唯一变量。")

# 3. 筛选需要的年份
target_years = ['2010', '2015', '2020']
available_years = [col for col in target_years if col in df.columns]

# 4. 分组统计每个 Variable 的缺失率
group = df.groupby('Variable')[available_years]
missing_stats = group.apply(lambda x: x.isna().sum() / len(x)).reset_index()

# 5. 重命名列
missing_stats = missing_stats.rename(columns={col: f"{col}_missing_rate" for col in available_years})

# 6. 保存结果
output_path = r'C:\Users\phc\Desktop\Variable_missing_rates.csv'
missing_stats.to_csv(output_path, index=False)

print(f"缺失率统计结果已保存到: {output_path}")

Variable 列中总共有 1028 个唯一变量。
缺失率统计结果已保存到: C:\Users\phc\Desktop\Variable_missing_rates.csv


### 模型情景数量情况

In [6]:
import pandas as pd

# 1. 读取数据
file_path = r'C:\Users\phc\Desktop\IamDataFrame.csv'
df = pd.read_csv(file_path)

# 2. 统计每个 Model 对应的唯一 Scenario 数量
model_scenario_counts = df.groupby('Model')['Scenario'].nunique().reset_index()
model_scenario_counts = model_scenario_counts.rename(columns={'Scenario': 'Scenario_Count'})

# 3. 保存结果为 CSV 文件
output_path = r'C:\Users\phc\Desktop\Model_Scenario_Count.csv'
model_scenario_counts.to_csv(output_path, index=False)

print(f"统计结果已保存到: {output_path}")

统计结果已保存到: C:\Users\phc\Desktop\Model_Scenario_Count.csv


### 每个模型在每年的缺失率

In [8]:
import pandas as pd

# 1. 读取数据
file_path = r'C:\Users\phc\Desktop\IamDataFrame2010_2100.csv'
df = pd.read_csv(file_path)

# 2. 年份列（2010-2100，每5年）
year_columns = [str(year) for year in range(2010, 2101, 5)]
available_years = [col for col in year_columns if col in df.columns]

# 3. 缺失率计算：每个 Model 对每一列（年份）的缺失数 ÷ 总行数
model_group = df.groupby('Model')
missing_rate = model_group[available_years].apply(lambda x: x.isna().sum() / len(x)).reset_index()

# 4. 列重命名：添加 _missing_rate 后缀
missing_rate = missing_rate.rename(columns={col: f"{col}_missing_rate" for col in available_years})

# 5. 保存结果为 CSV
output_path = r'C:\Users\phc\Desktop\Model_Yearly_Missing_Rate.csv'
missing_rate.to_csv(output_path, index=False)

print(f"缺失率统计结果已保存到: {output_path}")

缺失率统计结果已保存到: C:\Users\phc\Desktop\Model_Yearly_Missing_Rate.csv


## 数据处理

### 中短期样本集中存在的2020年后（不包含2020年）样本值的缺失问题，使用线性插值平滑

In [9]:
import pandas as pd

# 1. 读取 Excel 文件
file_path = r'C:\Users\phc\Desktop\Middle_short_model.xlsx'
df = pd.read_excel(file_path)

# 2. 定义年份列：仅考虑2025~2050年之间的插值
year_columns = [str(year) for year in range(2025, 2051, 5)]
available_years = [col for col in year_columns if col in df.columns]

# 3. 仅对这些列做线性插值（按行）
# 保证插值只作用于2025-2050范围内的缺失值
df[available_years] = df[available_years].transpose().interpolate(method='linear', axis=0).transpose()

# 4. 保存处理后的数据
output_path = r'C:\Users\phc\Desktop\Middle_short_model_interpolated.xlsx'
df.to_excel(output_path, index=False)

print(f"线性插值完成，文件已保存至: {output_path}")

线性插值完成，文件已保存至: C:\Users\phc\Desktop\Middle_short_model_interpolated.xlsx


## 模型变量平衡性匹配

### 模型包含的变量删除

In [13]:
import pandas as pd

# 1. 读取数据
file_path = r'C:\Users\phc\Desktop\Middle_short_model.xlsx'
df = pd.read_excel(file_path)

# 2. 获取每个模型的唯一变量列表
model_var_map = df.groupby('Model')['Variable'].unique()

# 3. 获取每个模型的变量数量
model_var_count = model_var_map.apply(len)

# 4. 创建对齐的变量 DataFrame
max_length = model_var_map.apply(len).max()
model_var_matrix = pd.DataFrame({model: pd.Series(variables) for model, variables in model_var_map.items()})

# 5. 在模型变量列表前添加一行变量数量
# 先将变量数构建为 DataFrame 并与变量列表 DataFrame 拼接
count_row = pd.DataFrame([model_var_count])
full_matrix = pd.concat([count_row, model_var_matrix], ignore_index=True)

# 6. 保存结果为 Excel 文件
output_path = r'C:\Users\phc\Desktop\Model_Variable_Matrix.xlsx'
full_matrix.to_excel(output_path, index=False)

print(f"模型变量对齐表（含变量数量）已保存到: {output_path}")

模型变量对齐表（含变量数量）已保存到: C:\Users\phc\Desktop\Model_Variable_Matrix.xlsx


### 变量包含的模型数量删除

In [15]:
import pandas as pd

# 1. 读取 Excel 数据
file_path = r'C:\Users\phc\Desktop\Middle_short_model_interpolated.xlsx'
df = pd.read_excel(file_path)

# 2. 提取唯一 Variable-Model 组合
var_model_df = df[['Variable', 'Model']].drop_duplicates().sort_values(['Variable', 'Model'])

# 3. 统计每个 Variable 出现的 Model 数量
model_counts = var_model_df.groupby('Variable')['Model'].nunique().reset_index()
model_counts = model_counts.rename(columns={'Model': 'Model_Count'})

# 4. 合并回长格式表格
result_df = pd.merge(var_model_df, model_counts, on='Variable')

# 5. 保存结果
output_path = r'C:\Users\phc\Desktop\Variable_Model_Long_With_Count.xlsx'
result_df.to_excel(output_path, index=False)

print(f"变量-模型长格式表（含模型数量）已保存至: {output_path}")

变量-模型长格式表（含模型数量）已保存至: C:\Users\phc\Desktop\Variable_Model_Long_With_Count.xlsx


In [16]:
import pandas as pd

# 1. 读取 Excel 文件
file_path = r'C:\Users\phc\Desktop\Variable contain model.xlsx'
df = pd.read_excel(file_path)

# 2. 提取唯一 Variable - Model 对
var_model_df = df[['Variable', 'Model']].drop_duplicates().sort_values(['Variable', 'Model'])

# 3. 统计每个 Variable 包含的 Model 数量
model_counts = var_model_df.groupby('Variable')['Model'].nunique().reset_index()
model_counts = model_counts.rename(columns={'Model': 'Model_Count'})

# 4. 合并模型数量信息
result_df = pd.merge(var_model_df, model_counts, on='Variable')

# 5. 保存结果
output_path = r'C:\Users\phc\Desktop\Variable_Model_Count_Long_Format.xlsx'
result_df.to_excel(output_path, index=False)

print(f"变量-模型统计（长格式）结果已保存至: {output_path}")


变量-模型统计（长格式）结果已保存至: C:\Users\phc\Desktop\Variable_Model_Count_Long_Format.xlsx


In [17]:
import pandas as pd

# 1. 读取 Excel 文件
file_path = r'C:\Users\phc\Desktop\删除的变量.xlsx'
df = pd.read_excel(file_path)

# 2. 获取唯一变量名称列表（去重）
unique_variables = df['Variable'].dropna().unique()

# 3. 转换为 DataFrame
result_df = pd.DataFrame({'Variable': sorted(unique_variables)})

# 4. 保存为 Excel 文件
output_path = r'C:\Users\phc\Desktop\要删除的唯一变量统计.xlsx'
result_df.to_excel(output_path, index=False)

print(f"要删除的唯一变量名称列表已保存至: {output_path}")


要删除的唯一变量名称列表已保存至: C:\Users\phc\Desktop\要删除的唯一变量统计.xlsx


In [18]:
import pandas as pd

# 1. 读取主数据集（包含变量）
main_file = r'C:\Users\phc\Desktop\Middle_short_model_interpolated.xlsx'
df_main = pd.read_excel(main_file)

# 2. 读取待删除变量列表
delete_file = r'C:\Users\phc\Desktop\要删除的唯一变量统计.xlsx'
df_delete = pd.read_excel(delete_file)

# 3. 获取要删除的变量名称集合（去重+非空）
delete_variables = set(df_delete['Variable'].dropna().unique())

# 4. 删除主表中这些变量对应的行
df_filtered = df_main[~df_main['Variable'].isin(delete_variables)]

# 5. 保存结果为新 Excel 文件
output_path = r'C:\Users\phc\Desktop\Middle_short_model_filtered.xlsx'
df_filtered.to_excel(output_path, index=False)

print(f"指定变量已删除，结果文件已保存至: {output_path}")

指定变量已删除，结果文件已保存至: C:\Users\phc\Desktop\Middle_short_model_filtered.xlsx


# 随机森林归因&计量检验

## 合并AR6和CD-link

In [2]:
import pandas as pd
import os

# 输入文件路径
file1 = r"C:\Users\phc\Desktop\AR6_data\excel_data\AR6_Scenarios_Database_R10_regions_v1.1.csv\AR6_Scenarios_Database_R10_regions_v1.1.csv"
file2 = r"C:\Users\phc\Desktop\CD-link\snapshot_all_regions_iamc_cdlinks.csv"

# 输出文件路径
output_folder = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验"
output_file = os.path.join(output_folder, "IamDataFrame2010_2100_world.csv")

# 读取两个数据集
df1 = pd.read_csv(file1, low_memory=False)
df2 = pd.read_csv(file2, low_memory=False)

# 外连接合并
df_merged = pd.merge(df1, df2, how="outer")

# 保存合并后的数据
df_merged.to_csv(output_file, index=False, encoding="utf-8-sig")


## 模型情景数量情况

In [4]:
import pandas as pd

# 1. 读取数据（添加 encoding 参数）
file_path = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world.csv"
df = pd.read_csv(file_path, encoding='gbk')  # 如果仍报错可尝试 encoding='ISO-8859-1'

# 2. 统计每个 Model 对应的唯一 Scenario 数量
model_scenario_counts = df.groupby('Model')['Scenario'].nunique().reset_index()
model_scenario_counts = model_scenario_counts.rename(columns={'Scenario': 'Scenario_Count'})

# 3. 保存结果为 CSV 文件
output_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Model_Scenario_Count_world.csv'
model_scenario_counts.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"统计结果已保存到: {output_path}")

统计结果已保存到: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Model_Scenario_Count_world.csv


## 删除情景缺失多的模型

In [6]:
import pandas as pd
import os

# 输入与输出路径设置
input_file = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world.csv"
output_file = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world_scenario.csv"

# 要删除的模型名称列表
models_to_remove = [
    "GAINS-dw4",
    "HEB 2.0",
    "ALADIN 1.0",
    "GCAM-USA CD-LINKS",
    "GENeSYS-MOD 2.9",
    "GMM-17",
    "BLUES-Brazil 1.0",
    "DNE21+ V.ADVANCE",
    "GCAM-USA 5.3",
    "IIASAPOP 2.0",
    "DNE21+ V.14",
    "DNE21+ V.14 (national)",
    "AIM/Enduse 3.0",
    "AIM/Enduse[Japan]"
]

# 读取数据
df = pd.read_csv(input_file, encoding='gbk', low_memory=False)

# 删除 Model 列中属于 models_to_remove 的行
df_filtered = df[~df['Model'].isin(models_to_remove)]

# 保存结果
df_filtered.to_csv(output_file, index=False, encoding='utf-8-sig')

## 删除2050年为缺失值的行

In [7]:
import pandas as pd

# 输入和输出路径
input_file = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world_scenario.csv"
output_file = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world_scenario_cleaned.csv"

# 读取文件（指定编码，如前遇到问题可尝试 'ISO-8859-1' 或 'gbk'）
df = pd.read_csv(input_file, encoding='utf-8', low_memory=False)

# 删除“2050”列值为缺失的数据行
df_cleaned = df.dropna(subset=['2050'])

# 保存结果
df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

## 2020年后（不包含2020年）样本值的缺失问题，使用线性插值平滑

In [8]:
import pandas as pd

# 1. 读取 CSV 文件（注意不是 Excel 文件）
file_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\IamDataFrame2010_2050_world_scenario_cleaned.csv'
df = pd.read_csv(file_path, encoding='utf-8', low_memory=False)

# 2. 定义年份列：仅考虑2025~2050年之间的插值
year_columns = [str(year) for year in range(2025, 2051, 5)]
available_years = [col for col in year_columns if col in df.columns]

# 3. 仅对这些列做线性插值（按行）
df[available_years] = df[available_years].transpose().interpolate(method='linear', axis=0).transpose()

# 4. 保存处理后的数据到 Excel 文件
output_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_world.xlsx'
df.to_excel(output_path, index=False)

print(f"线性插值完成，文件已保存至: {output_path}")

线性插值完成，文件已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_world.xlsx


## 模型包含的变量统计

In [10]:
import pandas as pd

# 1. 读取数据
file_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_world.xlsx'
df = pd.read_excel(file_path)

# 2. 获取每个模型的唯一变量列表
model_var_map = df.groupby('Model')['Variable'].unique()

# 3. 获取每个模型的变量数量
model_var_count = model_var_map.apply(len)

# 4. 创建对齐的变量 DataFrame
max_length = model_var_map.apply(len).max()
model_var_matrix = pd.DataFrame({model: pd.Series(variables) for model, variables in model_var_map.items()})

# 5. 在模型变量列表前添加一行变量数量
count_row = pd.DataFrame([model_var_count])
full_matrix = pd.concat([count_row, model_var_matrix], ignore_index=True)

# 6. 保存结果为 Excel 文件
output_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Model_Variable_Matrix_world.xlsx'
full_matrix.to_excel(output_path, index=False)

print(f"模型变量对齐表（含变量数量）已保存到: {output_path}")

模型变量对齐表（含变量数量）已保存到: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Model_Variable_Matrix_world.xlsx


## 模型直接删除到与Middle_short_model_filtered.xlsx一样

In [11]:
import pandas as pd

# 输入文件路径
input_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_world.xlsx'

# 输出文件路径
output_path = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_selected_models.xlsx'

# 要保留的模型列表
models_to_keep = [
    "AIM/CGE 2.1", "AIM/CGE 2.2", "C3IAM 1.0", "C3IAM 2.0", "COFFEE 1.1",
    "COPPE-COFFEE 1.0", "GCAM 4.0", "GCAM 5.2", "GCAM 5.3", "GCAM-PR 5.3",
    "GCAM5.2_NET", "GEM-E3 V1", "GEM-E3_V2021", "GEMINI-E3 7.0",
    "IMACLIM 1.1", "IMAGE 3.0", "IMAGE 3.0.1", "IMAGE 3.2"
]

# 读取 Excel 文件
df = pd.read_excel(input_path)

# 筛选指定模型
filtered_df = df[df['Model'].isin(models_to_keep)]

# 保存结果
filtered_df.to_excel(output_path, index=False)

print(f"筛选后的模型数据已保存至: {output_path}")

筛选后的模型数据已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_selected_models.xlsx


## 变量包含的模型数量删除，删除了包含模型家族（无版本迭代）数量少于4个的变量

### 统计每个变量包含的模型数量

In [12]:
import pandas as pd

# 输入文件路径
input_file = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_selected_models.xlsx'

# 输出文件路径
output_file = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Variable_Model_Count.xlsx'

# 读取数据
df = pd.read_excel(input_file)

# 按 Variable 分组，统计唯一的 Model 数量
variable_model_counts = df.groupby('Variable')['Model'].nunique().reset_index()
variable_model_counts.columns = ['Variable', 'Unique_Model_Count']

# 保存结果
variable_model_counts.to_excel(output_file, index=False)

print(f"每个变量对应的唯一模型数量已保存到: {output_file}")


每个变量对应的唯一模型数量已保存到: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Variable_Model_Count.xlsx


### 删除包含模型家族少于4的变量

In [13]:
import pandas as pd

# 输入文件路径
main_file = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_interpolated_selected_models.xlsx'
remove_file = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\var_deleted.xlsx'

# 输出文件路径（更新为你指定的文件名）
output_file = r'C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_filtered_world.xlsx'

# 读取主数据和待删除变量列表
df_main = pd.read_excel(main_file)
df_remove = pd.read_excel(remove_file)

# 获取待删除的变量名列表
variables_to_delete = df_remove['Variable'].unique()

# 删除匹配的变量
df_filtered = df_main[~df_main['Variable'].isin(variables_to_delete)]

# 保存结果
df_filtered.to_excel(output_file, index=False)

print(f"筛选后的数据已保存到: {output_file}")


筛选后的数据已保存到: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\1_数据来源及清洗处理\data_processing\随机森林_计量检验\Middle_short_model_filtered_world.xlsx
